## Building basic likelihoods

In [1]:
import sys
import os
import glob

import tqdm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

%matplotlib inline

In [3]:
import jax
jax.config.update('jax_enable_x64', True)

import jax.random
import jax.numpy as jnp

In [4]:
import discovery as ds
import discovery.models.nanograv as ds_nanograv
import discovery.samplers.numpyro as ds_numpyro

Read nanograv pulsars

In [5]:
allpsrs = [ds.Pulsar.read_feather(psrfile) for psrfile in sorted(glob.glob('../data/*-[JB]*.feather'))]

In [6]:
psr = allpsrs[0]

### Single pulsar likelihood

#### Measurement noise only, no backends

In [8]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement_simple(psr)])

What are the active parameters?

In [9]:
m.logL.params

['B1855+09_efac', 'B1855+09_log10_t2equad']

Sample random values from their priors

In [9]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_efac': 0.9068382830056773,
 'B1855+09_log10_t2equad': -7.681411506369306}

Evaluate the likelihood

In [11]:
m.logL(p0)

Array(88836.8723052, dtype=float64)

Try compiled version, grad

In [18]:
jax.jit(m.logL)(p0)

Array(89964.27597335, dtype=float64)

In [19]:
jax.grad(m.logL)(p0)

{'B1855+09_efac': Array(25210.26046491, dtype=float64, weak_type=True),
 'B1855+09_log10_t2equad': Array(250.88301607, dtype=float64, weak_type=True)}

#### Measurement noise only, nanograv backends, free parameters

In [14]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr)])

In [15]:
m.logL.params

['B1855+09_430_ASP_efac',
 'B1855+09_430_ASP_log10_t2equad',
 'B1855+09_430_PUPPI_efac',
 'B1855+09_430_PUPPI_log10_t2equad',
 'B1855+09_L-wide_ASP_efac',
 'B1855+09_L-wide_ASP_log10_t2equad',
 'B1855+09_L-wide_PUPPI_efac',
 'B1855+09_L-wide_PUPPI_log10_t2equad']

In [17]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_430_ASP_efac': 1.0458700396876013,
 'B1855+09_430_ASP_log10_t2equad': -7.775526234972234,
 'B1855+09_430_PUPPI_efac': 1.0142689714595372,
 'B1855+09_430_PUPPI_log10_t2equad': -7.477477697510203,
 'B1855+09_L-wide_ASP_efac': 0.9509780051012473,
 'B1855+09_L-wide_ASP_log10_t2equad': -5.735913773186028,
 'B1855+09_L-wide_PUPPI_efac': 1.043962211974928,
 'B1855+09_L-wide_PUPPI_log10_t2equad': -6.489535802684073}

In [18]:
m.logL(p0)

Array(98829.22637937, dtype=float64)

#### Measurement noise only, nanograv backends, parameters from noisedict

In [25]:
psr.noisedict

{'B1855+09_430_ASP_efac': 1.115935306813982,
 'B1855+09_430_ASP_log10_t2equad': -7.564164330699591,
 'B1855+09_430_PUPPI_efac': 1.000049037085653,
 'B1855+09_430_PUPPI_log10_t2equad': -6.572540211467256,
 'B1855+09_L-wide_ASP_efac': 1.043114017270374,
 'B1855+09_L-wide_ASP_log10_t2equad': -6.517929916655293,
 'B1855+09_L-wide_PUPPI_efac': 1.1118432332882,
 'B1855+09_L-wide_PUPPI_log10_t2equad': -7.755603780476984,
 'B1855+09_430_ASP_log10_ecorr': -6.798122106550257,
 'B1855+09_430_PUPPI_log10_ecorr': -5.6989064141929715,
 'B1855+09_L-wide_ASP_log10_ecorr': -6.120457109433745,
 'B1855+09_L-wide_PUPPI_log10_ecorr': -6.641667916624413,
 'B1855+09_red_noise_log10_A': -13.940953818371378,
 'B1855+09_red_noise_gamma': -3.68432133461766}

In [26]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict)])

In [27]:
m.logL.params

[]

In [28]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(96686.81011373615)

#### Add ECORR noise (GP), free params

In [65]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr),
                         ds.makegp_ecorr(psr)])

In [66]:
m.logL.params

['B1855+09_430_ASP_efac',
 'B1855+09_430_ASP_log10_ecorr',
 'B1855+09_430_ASP_log10_t2equad',
 'B1855+09_430_PUPPI_efac',
 'B1855+09_430_PUPPI_log10_ecorr',
 'B1855+09_430_PUPPI_log10_t2equad',
 'B1855+09_L-wide_ASP_efac',
 'B1855+09_L-wide_ASP_log10_ecorr',
 'B1855+09_L-wide_ASP_log10_t2equad',
 'B1855+09_L-wide_PUPPI_efac',
 'B1855+09_L-wide_PUPPI_log10_ecorr',
 'B1855+09_L-wide_PUPPI_log10_t2equad']

In [67]:
m.logL(ds.sample_uniform(m.logL.params))

Array(95422.18669251, dtype=float64)

#### Add ECORR noise (GP), noisedict params

In [33]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict)])

In [34]:
m.logL.params

[]

In [35]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(103037.9457516601)

#### Add timing model

In [36]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True)])

In [38]:
m.logL.params

[]

In [39]:
m.logL(ds.sample_uniform(m.logL.params))

np.float64(94817.46786020289)

#### Add red noise (powerlaw)

In [12]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')])

In [13]:
m.logL.params

['B1855+09_rednoise_gamma', 'B1855+09_rednoise_log10_A']

In [14]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 0.9133990157107308,
 'B1855+09_rednoise_log10_A': -11.886187005599341}

In [15]:
m.logL(p0)

Array(91001.08284231, dtype=float64)

#### Add red noise (powerlaw, fixed gamma)

In [54]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.partial(ds.powerlaw, gamma=4.33), components=30, name='rednoise')])

In [55]:
m.logL.params

['B1855+09_rednoise_log10_A']

In [56]:
m.logL(ds.sample_uniform(m.logL.params))

Array(94818.95650736, dtype=float64)

#### Add red noise (free spectrum)

In [16]:
m = ds.PulsarLikelihood([psr.residuals,
                         ds.makenoise_measurement(psr, psr.noisedict),
                         ds.makegp_ecorr(psr, psr.noisedict),
                         ds.makegp_timing(psr, svd=True),
                         ds.makegp_fourier(psr, ds.freespectrum, components=30, name='rednoise')])

In [17]:
m.logL.params

['B1855+09_rednoise_log10_rho(30)']

In [31]:
p0 = {'B1855+09_rednoise_log10_rho(30)': 1e-2 * np.random.randn(30)}; p0

In [32]:
m.logL(p0)

Array(90205.33814993, dtype=float64)

### Multiple pulsars

In [34]:
psrs = allpsrs[:3]

#### Combined likelihood

In [35]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')])
                        for psr in psrs])

In [36]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A']

In [37]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 2.13401299874643,
 'B1855+09_rednoise_log10_A': -17.65027485398618,
 'B1937+21_rednoise_gamma': 0.0016059734948359328,
 'B1937+21_rednoise_log10_A': -19.23839193615695,
 'B1953+29_rednoise_gamma': 0.14528115292650945,
 'B1953+29_rednoise_log10_A': -11.216503664115905}

In [38]:
m.logL(p0)

Array(428610.75808803, dtype=float64)

#### Add common noise

Indicating parameters under common shares them among pulsars

In [39]:
T = ds.getspan(psrs)

In [40]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, T=T, name='rednoise'),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=14, T=T, name='crn',
                                                               common=['crn_log10_A', 'crn_gamma'])])
                        for psr in psrs])

In [41]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [ ]:
p0 = ds.sample_uniform(m.logL.params); p0

In [42]:
p0 = {'B1855+09_rednoise_gamma': 0.22692837299032842,
 'B1855+09_rednoise_log10_A': -14.855929802663269,
 'B1937+21_rednoise_gamma': 1.0599042474713591,
 'B1937+21_rednoise_log10_A': -15.116911973999551,
 'B1953+29_rednoise_gamma': 3.5757301091084486,
 'B1953+29_rednoise_log10_A': -14.37084627399558,
 'crn_gamma': 4.432798514507736,
 'crn_log10_A': -13.879467862230983}

In [43]:
m.logL(p0)

Array(473878.04941861, dtype=float64)

#### Parallelize red components

Coordinated timespan is required

In [44]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = [ds.makecommongp_fourier(psrs, ds.powerlaw, components=30, T=T, name='rednoise'),
                                   ds.makecommongp_fourier(psrs, ds.powerlaw, components=14, T=T, name='crn',
                                                           common=['crn_log10_A', 'crn_gamma'])])

In [45]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [48]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 6.2016324259132976,
 'B1855+09_rednoise_log10_A': -16.51182106267872,
 'B1937+21_rednoise_gamma': 5.417321744296163,
 'B1937+21_rednoise_log10_A': -14.81401491518855,
 'B1953+29_rednoise_gamma': 5.802346787367874,
 'B1953+29_rednoise_log10_A': -18.89307111376218,
 'crn_gamma': 6.518869741837323,
 'crn_log10_A': -11.52173724907908}

In [49]:
m.logL(p0)

Array(473520.64257291, dtype=float64)

#### Reuse Fourier vectors

`ds.makepowerlaw_crn` yields the sum of two powerlaws, with possibly different number of components.

In [50]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = ds.makecommongp_fourier(psrs, ds.makepowerlaw_crn(components=14), components=30, T=T, name='rednoise',
                                                          common=['crn_log10_A', 'crn_gamma']))

In [51]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'crn_gamma',
 'crn_log10_A']

In [52]:
m.logL(p0)

Array(473520.64257291, dtype=float64)

#### Add global spatially correlated process

Note `ds.makeglobalgp_fourier` requires the ORF, but not the `common` specification, which is automatic.

In [61]:
m = ds.ArrayLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True)]) for psr in psrs],
                       commongp = ds.makecommongp_fourier(psrs, ds.powerlaw, components=30, T=T, name='rednoise'),
                       globalgp = ds.makeglobalgp_fourier(psrs, ds.powerlaw, ds.hd_orf, components=14, T=T, name='gw'))

In [54]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_gamma',
 'gw_log10_A']

In [57]:
p0 = ds.sample_uniform(m.logL.params); p0

{'B1855+09_rednoise_gamma': 6.140201729456801,
 'B1855+09_rednoise_log10_A': -12.7195973990503,
 'B1937+21_rednoise_gamma': 0.06606923103279372,
 'B1937+21_rednoise_log10_A': -17.497945172565075,
 'B1953+29_rednoise_gamma': 1.4612722661200002,
 'B1953+29_rednoise_log10_A': -19.093990582967415,
 'gw_gamma': 2.7598567422273215,
 'gw_log10_A': -13.922241226263383}

In [62]:
m.logL(p0)

Array(473547.1319047, dtype=float64)

#### Another way of doing this (useful if variable GPs differ among pulsars)

In [63]:
m = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                             ds.makenoise_measurement(psr, psr.noisedict),
                                             ds.makegp_ecorr(psr, psr.noisedict),
                                             ds.makegp_timing(psr, svd=True),
                                             ds.makegp_fourier(psr, ds.powerlaw, components=30, name='rednoise')]) for psr in psrs],
                        globalgp = ds.makeglobalgp_fourier(psrs, ds.powerlaw, ds.hd_orf, components=14, T=T, name='gw'))

In [59]:
m.logL.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_gamma',
 'gw_log10_A']

In [64]:
m.logL(p0)

Array(473547.35835492, dtype=float64)